In [2]:
!pip install tensorflow
!pip install opencv-python
!pip install cmake

  Using cached absl_py-2.1.0-py3-none-any.whl.metadata (2.3 kB)
  Using cached astunparse-1.6.3-py2.py3-none-any.whl.metadata (4.4 kB)
  Using cached flatbuffers-24.3.25-py2.py3-none-any.whl.metadata (850 bytes)
  Using cached google_pasta-0.2.0-py3-none-any.whl.metadata (814 bytes)
  Using cached libclang-18.1.1-py2.py3-none-win_amd64.whl.metadata (5.3 kB)
  Using cached opt_einsum-3.3.0-py3-none-any.whl.metadata (6.5 kB)
  Using cached termcolor-2.4.0-py3-none-any.whl.metadata (6.1 kB)
     ---------------------------------------- 0.0/61.0 kB ? eta -:--:--
     --------------------------------- ------ 51.2/61.0 kB 2.7 MB/s eta 0:00:01
     --------------------------------- ------ 51.2/61.0 kB 2.7 MB/s eta 0:00:01
     --------------------------------- ------ 51.2/61.0 kB 2.7 MB/s eta 0:00:01
     -------------------------------------- 61.0/61.0 kB 406.4 kB/s eta 0:00:00
  Using cached wheel-0.44.0-py3-none-any.whl.metadata (2.3 kB)
  Using cached rich-13.7.1-py3-none-any.whl.metadata


[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


   -------------- ------------------------- 141.5/385.2 MB 5.1 MB/s eta 0:00:48
   -------------- ------------------------- 141.8/385.2 MB 5.1 MB/s eta 0:00:48
   -------------- ------------------------- 141.9/385.2 MB 5.0 MB/s eta 0:00:49
   -------------- ------------------------- 142.3/385.2 MB 5.0 MB/s eta 0:00:49
   -------------- ------------------------- 142.5/385.2 MB 5.0 MB/s eta 0:00:49
   -------------- ------------------------- 142.6/385.2 MB 4.9 MB/s eta 0:00:50
   -------------- ------------------------- 142.7/385.2 MB 4.8 MB/s eta 0:00:51
   -------------- ------------------------- 142.8/385.2 MB 4.8 MB/s eta 0:00:51
   -------------- ------------------------- 142.9/385.2 MB 4.7 MB/s eta 0:00:52
   -------------- ------------------------- 143.0/385.2 MB 4.6 MB/s eta 0:00:53
   -------------- ------------------------- 143.1/385.2 MB 4.6 MB/s eta 0:00:53
   -------------- ------------------------- 143.2/385.2 MB 4.5 MB/s eta 0:00:54
   -------------- ----------------------


[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


  Using cached opencv_python-4.10.0.84-cp37-abi3-win_amd64.whl.metadata (20 kB)
Using cached opencv_python-4.10.0.84-cp37-abi3-win_amd64.whl (38.8 MB)
   ---------------------------------------- 0.0/35.6 MB ? eta -:--:--
   ---------------------------------------- 0.4/35.6 MB 11.8 MB/s eta 0:00:03
    --------------------------------------- 0.7/35.6 MB 10.7 MB/s eta 0:00:04
    --------------------------------------- 0.8/35.6 MB 8.9 MB/s eta 0:00:04
    --------------------------------------- 0.8/35.6 MB 8.9 MB/s eta 0:00:04
    --------------------------------------- 0.8/35.6 MB 8.9 MB/s eta 0:00:04
    --------------------------------------- 0.8/35.6 MB 8.9 MB/s eta 0:00:04
    --------------------------------------- 0.8/35.6 MB 8.9 MB/s eta 0:00:04
    --------------------------------------- 0.8/35.6 MB 8.9 MB/s eta 0:00:04
    --------------------------------------- 0.8/35.6 MB 8.9 MB/s eta 0:00:04
    --------------------------------------- 0.8/35.6 MB 8.9 MB/s eta 0:00:04
    ---


[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import tensorflow as tf
import cv2
import numpy as np
import cmake
import pickle

In [3]:
from tensorflow.keras.applications import EfficientNetB2
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout, Input
from tensorflow.keras.models import Model

def get_model():
    input_layer = Input(shape=(96, 96, 3))
    backbone = EfficientNetB2(include_top=False, weights=None, input_tensor=input_layer)
    x = GlobalAveragePooling2D()(backbone.output)
    x = Dropout(0.3)(x)
    x = Dense(128, activation="relu")(x)
    output_layer = Dense(7, activation="softmax")(x)
    model = Model(inputs=input_layer, outputs=output_layer)
    return model


In [4]:
model = get_model()

In [5]:
import os

file_path = r'C:\Users\DEEPAK\Downloads\best_model.weights.h5'
if os.path.isfile(file_path):
    model.load_weights(file_path)
else:
    print(f"File not found: {file_path}")

In [6]:
sample_obj = {'key': 'value'}
with open(r'C:\Users\DEEPAK\Downloads\label_encoder.pkl', 'wb') as f:
    pickle.dump(sample_obj, f)

In [6]:
#def load_obj(file_path):
#   if not os.path.isfile(file_path):
#        raise FileNotFoundError(f"No such file: '{file_path}'")
    
#    if os.path.getsize(file_path) == 0:
#        raise EOFError(f"File is empty: '{file_path}'")

#    with open(file_path, 'rb') as pickle_obj:
#        obj = pickle.load(pickle_obj)
#    return obj

#file_path = r"C:\Users\DEEPAK\Downloads\label_encoder.pkl"
#Le = load_obj(file_path)

In [7]:
def ProcessImage(image):
    if image.ndim == 2:
        image = tf.expand_dims(image, -1)
    image = tf.convert_to_tensor(image)
    image = tf.image.resize(image, [96,96], method="bilinear")
    image = tf.expand_dims(image, 0)
    return image    
def RealtimePrediction (image, model, encoder_):
    prediction = model.predict(image)
    prediction = np.argmax(prediction, axis = 1)
    return encoder_[prediction[0]]
    #return encoder_.inverse_transform[prediction[0]]
def rec_to_bb(rect):
    x = rect.left()
    y = rect.top()
    w = rect.right() - x
    z = rect.bottom() -y
    return (x ,y, w, z)

In [1]:
#import dlib
#videocapture = cv2.VideoCapture(0)
#detector = dlib.get_frontal_face_detector()

#while True:
#    ret, frame = videocapture.read()
#    if not ret:
#         break
#    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
#    rects = detector(gray, 0)
#    if len(rects) >=1:
#        for rect in rects:
#            (x, y, w, h) = rec_to_bb(rect)
#            img = gray[y-10 : y+h+10, x-10 : x+w+10]
#            if img.ndim == 3 and img.shape[-1] == 3:
#            #if img.shape[0] == 0 or img.shape[1] ==0:
#                cv2.imshow("Frame", frame)
#            else:
#                img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
#                img = ProcessImage(img)
#                out = RealtimePrediction(img, model, Le)
#                cv2.rectangle(frame, (x, y), (x+w, y+h), (0,255,0), 0)
#                z = y-15 if y-15>15 else y+15
#                cv2.putText(frame, str(out), (x,z), cv2.FONT_HERSHEY_SIMPLEX, 0.75, (0, 255, 0), 2)
#        cv2.imshow("Frame", frame)    
#    else:
#        cv2.imshow("Frame", frame)
#    if cv2.waitKey(1) & 0xFF == ord('q'):
#        break
#VideoCapture.release()
#cv2.destroyAllwindows()

In [8]:
import dlib
videocapture = cv2.VideoCapture(0)
detector = dlib.get_frontal_face_detector()
Le = {0: 'angry', 1: 'disgust', 2: 'fear',3: 'happy',4: 'neutral',5: 'sad',6: 'surprise'}
while True:
    ret, frame = videocapture.read()
    if not ret:
         break
    
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    rects = detector(gray, 0)

    if len(rects) >= 1:
        for rect in rects:
            (x, y, w, h) = rec_to_bb(rect)
            img = gray[y-10 : y+h+10, x-10 : x+w+10]
            
            # Process the image
            img = ProcessImage(img)
            out = RealtimePrediction(img, model, Le)
            
            # Draw the rectangle and label on the frame
            cv2.rectangle(frame, (x, y), (x+w, y+h), (0,255,0), 2)
            z = y-15 if y-15>15 else y+15
            cv2.putText(frame, str(out), (x, z), cv2.FONT_HERSHEY_SIMPLEX, 0.75, (0, 255, 0), 2)

        cv2.imshow("Frame", frame)    
    else:
        cv2.imshow("Frame", frame)
    
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

videocapture.release()
cv2.destroyAllWindows()

1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 61ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 67ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 64ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 71ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 59ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 59ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 62ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/step
1/1 ━━━━━━━━━━

InvalidArgumentError: {{function_node __wrapped__ResizeBilinear_device_/job:localhost/replica:0/task:0/device:CPU:0}} input image must be of non-zero size [Op:ResizeBilinear] name: 

In [12]:
pip install dlib


  Using cached dlib-19.24.5.tar.gz (3.3 MB)
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Created wheel for dlib: filename=dlib-19.24.5-cp312-cp312-win_amd64.whl size=2884263 sha256=075468862396f05075d1dc8ea726725f8e14e19b5d30fef5e5daf42605cb44a3
  Stored in directory: c:\users\deepak\appdata\local\pip\cache\wheels\51\44\9c\0cd48b15210a320d151e0a77e728dca1c2ad6678e913510d56
Successfully built dlib



[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip
